In [37]:
import pandas as pd
import numpy as np
from pathlib import Path


# 1.Load data

In [52]:
annotations = pd.read_csv("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/results/HD_DIT_HAP/12_concatenated/annotations.tsv", sep="\t", index_col=[0,1,2,3])

ncRNAs_annotations = pd.read_csv("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/results/HD_DIT_HAP/19_insertion_in_non_coding_genes/annotated_insertions_in_non_coding_genes.tsv", sep="\t", index_col=[0,1,2,3])

ncRNAs_gene_level_statistics = pd.read_csv("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/results/HD_DIT_HAP/19_insertion_in_non_coding_genes/Non_coding_genes_Gene_level_statistics_fitted.tsv", sep="\t")

sysID2type = ncRNAs_annotations[["Systematic ID", "Feature"]].copy().reset_index(drop=True).drop_duplicates()
sysID2type = dict(zip(sysID2type["Systematic ID"], sysID2type["Feature"]))

ncRNAs_gene_level_statistics.insert(2, "Type", ncRNAs_gene_level_statistics["Systematic ID"].map(sysID2type))

# 2.ncRNAs only in non-coding genes

In [39]:
in_gene_insertion_index = annotations.query("Type != 'Intergenic region' and Distance_to_stop_codon > 4").index

IGR_index = annotations[~annotations.index.isin(in_gene_insertion_index)].index

In [41]:
ncRNA_in_coding_gene_insertion_counts = pd.DataFrame()

for sysID, gene_annotations in ncRNAs_annotations.groupby("Systematic ID"):

    insertions_in_current_ncRNA = annotations.loc[gene_annotations.index]

    insertions_in_both_current_ncRNA_and_coding_region = insertions_in_current_ncRNA.query("Type != 'Intergenic region' and Distance_to_stop_codon > 4")

    if len(insertions_in_both_current_ncRNA_and_coding_region) > 0:
        ncRNA_in_coding_gene_insertion_counts.loc[sysID, "Overlapping_with_coding_region"] = "Yes"
    else:
        ncRNA_in_coding_gene_insertion_counts.loc[sysID, "Overlapping_with_coding_region"] = "No"

In [44]:
ncRNA_in_coding_gene_insertion_counts.to_csv("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/results/HD_DIT_HAP/19_insertion_in_non_coding_genes/ncRNA_in_coding_gene_summary.tsv", sep="\t", index=True)

In [45]:
ncRNAs_only_in_non_coding_regions = ncRNA_in_coding_gene_insertion_counts.query("Overlapping_with_coding_region == 'No'").index.tolist()

# 3.Non-coding genes analysis

In [56]:
ncRNAs_gene_level_statistics_only_in_non_coding_regions = ncRNAs_gene_level_statistics.query("`Systematic ID` in @ncRNAs_only_in_non_coding_regions")

In [58]:
ncRNAs_gene_level_statistics_only_in_non_coding_regions.to_csv("/data/c/yangyusheng_optimized/DIT_HAP_pipeline/results/HD_DIT_HAP/19_insertion_in_non_coding_genes/Gene_level_statistics_of_ncRNAs_only_in_non_coding_regions.tsv", sep="\t", index=False)

In [61]:
ncRNAs_gene_level_statistics_only_in_non_coding_regions.query("A > 2")["Type"].value_counts()

Type
lncRNA    103
tRNA        3
snoRNA      1
snRNA       1
Name: count, dtype: int64

In [62]:
ncRNAs_gene_level_statistics_only_in_non_coding_regions

,Systematic ID,Name,Type,FYPOviability,DeletionLibrary_essentiality,time_points,Status,A,um,lam,...,t0_fitted,t1_fitted,t2_fitted,t3_fitted,t4_fitted,t0_residual,t1_residual,t2_residual,t3_residual,t4_residual
0,SPATRNAARG.03,SPATRNAARG.03,tRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,0.396,-0.028,1.386,...,0.049,0.015,0.001,0.000,0.000,-0.049,0.164,-0.023,-0.103,0.134
1,SPATRNAASP.01,SPATRNAASP.01,tRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.331,-0.056,-0.000,...,-0.022,-0.131,-0.268,-0.317,-0.328,0.022,-0.392,0.006,0.082,0.070
2,SPATRNAASP.02,SPATRNAASP.02,tRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.335,-0.056,-0.000,...,-0.022,-0.132,-0.270,-0.321,-0.332,0.022,-0.043,-0.060,0.096,-0.028
3,SPATRNAILE.02,SPATRNAILE.02,tRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,5.705,0.956,-0.000,...,0.376,2.248,4.608,5.464,5.652,-0.376,0.256,0.638,-0.742,0.120
4,SPATRNALEU.02,SPATRNALEU.02,tRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.006,0.001,1.181,...,-0.001,-0.000,-0.000,-0.000,-0.000,0.001,0.030,0.089,0.148,0.166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5857,SPSNORNA.49,snR97,snoRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.079,0.013,9.511,...,-0.076,-0.071,-0.050,-0.008,-0.000,0.076,-0.071,-0.015,-0.002,0.072
5858,SPSNORNA.50,snR98,snoRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.054,0.004,1.204,...,-0.006,-0.002,-0.000,-0.000,-0.000,0.006,-0.025,0.081,-0.091,0.225
5859,SPSNORNA.52,snR101,snoRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,-0.133,0.022,9.511,...,-0.128,-0.120,-0.084,-0.014,-0.000,0.128,-0.104,-0.035,-0.070,0.105
5860,SPSNRNA.02,snu2,snRNA,unknown,Not_determined,"[0.0, 2.352, 5.588, 9.104, 12.48]",Success,6.155,1.031,0.130,...,0.344,2.291,4.908,5.880,6.095,-0.344,-0.081,0.624,-0.675,0.198
